In [40]:
%%time
#load data

import pandas as pd 
import numpy as np
import time


n = 14
date = range(1,43)
high = [274,273.25,272,270.75,270,270.5,268.5,265.5,262.5,263.5,269.5,267.25,267.5,269.75,268.25,264,268,266,274,277.5,277,272,267.25,269.25,266,265,264.75,261,257.5,259,259.75,257.25,250,254.25,254,253.25,253.25,251.75,253,251.5,246.25,244.25]
low = [272,270.25,269.75,268,269,268,266.5,263,259,260,263,265,265.5,266,263.25,261.5,266.25,264.25,267,273.5,272.5,269.5,264,263,263.5,262,261.5,255.5,253,254,257.5,250,247,252.75,250.50,250.25,251,250.50,249.5,245.25,240,241.25]
close = [272.75,270.75,270,269.25,269.75,270,266.5,263.25,260.25,263,266.5,267,265.75,268.5,264.25,264,266.5,265.25,273,276.75,273,270.25,266.75,263,265.5,262.25,262.75,255.5,253,257.5,257.5,250,249.75,253.75,251.25,250.5,253,251.5,250,245.75,242.75,243.5]

data = {'date': date, 'high': high, 'low':low, 'close': close}


dataframe = pd.DataFrame(data)



CPU times: user 889 µs, sys: 0 ns, total: 889 µs
Wall time: 833 µs


In [41]:
%%timeit
##Original method validate by book

dataframe['tr1'] = dataframe.high - dataframe.low
dataframe['tr2'] = (dataframe.high - dataframe.close.shift(1)).abs()
dataframe['tr3'] = (dataframe.low - dataframe.close.shift(1)).abs()
dataframe['tr'] = dataframe[['tr1','tr2','tr3']].max(axis=1)
dataframe['plus_dm'] = 0
dataframe['minus_dm'] = 0
dataframe['delta_high'] = dataframe.high - dataframe.high.shift(1)
dataframe['delta_low'] = dataframe.low.shift(1) - dataframe.low
dataframe['delta_high'] = dataframe.delta_high.mask(dataframe.delta_high < 0,0)
dataframe['delta_low'] = dataframe.delta_low.mask(dataframe.delta_low < 0,0)
dataframe['plus_dm'] = dataframe.plus_dm.mask(dataframe.delta_high > dataframe.delta_low,dataframe.delta_high)
dataframe['minus_dm'] = dataframe.minus_dm.mask(dataframe.delta_low > dataframe.delta_high,dataframe.delta_low)

dataframe.at[0,'tr'] = np.nan
dataframe.at[0,'plus_dm'] = np.nan
dataframe.at[0,'minus_dm'] = np.nan

#print(dataframe[['date','high','low','close','tr','plus_dm','minus_dm']])

tr = pd.Series(dataframe.tr)
plus_dm = pd.Series(dataframe.plus_dm)
minus_dm = pd.Series(dataframe.minus_dm)
TRn = pd.Series(np.zeros(tr.size))
plus_DMn = pd.Series(dataframe.plus_dm)
minus_DMn = pd.Series(dataframe.minus_dm)

TRn[TRn.index[n-1]] = np.sum(tr[:n])
TRn = TRn.drop(TRn.index[:n-1])
 
plus_DMn[plus_DMn.index[n-1]] = np.sum(plus_dm[:n])
plus_DMn = plus_DMn.drop(plus_DMn.index[:n-1])

minus_DMn[minus_DMn.index[n-1]] = np.sum(minus_dm[:n])
minus_DMn = minus_DMn.drop(minus_DMn.index[:n-1])

for index,value in plus_DMn.items():
    if index >= n :
        plus_DMn[index] = (plus_DMn[index-1] - (plus_DMn[index-1]/14)) + plus_dm[index]
for index,value in minus_DMn.items():
    if index >= n :
        minus_DMn[index] = (minus_DMn[index-1] - (minus_DMn[index-1]/14)) + minus_dm[index]
for index,value in TRn.items():
    if index >= n :
        TRn[index] = (TRn[index-1] - (TRn[index-1]/14)) + tr[index]

dataframe['plus_DMn'] = plus_DMn
dataframe['minus_DMn'] = minus_DMn
dataframe['TRn'] = TRn

#print(dataframe[['date','tr','plus_dm','minus_dm','TRn','plus_DMn','minus_DMn']])

dataframe['plus_DI'] = dataframe.plus_DMn/dataframe.TRn
dataframe['minus_DI'] = dataframe.minus_DMn/dataframe.TRn

dataframe['TDM'] = abs(dataframe.plus_DI- dataframe.minus_DI)
dataframe['SUM_DM'] = dataframe.plus_DI + dataframe.minus_DI

dataframe['DX'] = dataframe['TDM'] / dataframe['SUM_DM']

#print(dataframe[['plus_DMn','minus_DMn','plus_DI','minus_DI','TDM','SUM_DM','DX']])

dx = pd.Series(dataframe['DX'])
adx = pd.Series(dataframe['DX'])
#print(adx[adx.index[n:2*n]])

adx[adx.index[(2*n) - 1]]= adx[adx.index[n:2*n]].mean()

adx = adx.drop(adx.index[:(2*n)-1])
for index,value in adx.items():
    if index >= 2*n:
        adx[index] = ( ((n-1)*adx[index-1]) + dx[index] )/n
dataframe['ADX'] = adx

dataframe['ADXR'] = (dataframe.ADX + dataframe.ADX.shift(n-1)) / 2
# en realidad deberia ser 14 no 13.

#print(dataframe[['TDM','SUM_DM','DX','ADX',"ADXR"]])

#dataframe.to_csv("original_final_dataframe_dm" +  str(int(time.time())) +  ".csv")

13.1 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%%timeit
##faster alternative method

dataframe['tr1'] = dataframe.high - dataframe.low
dataframe['tr2'] = (dataframe.high - dataframe.close.shift(1)).abs()
dataframe['tr3'] = (dataframe.low - dataframe.close.shift(1)).abs()
dataframe['tr'] = dataframe[['tr1','tr2','tr3']].max(axis=1)
dataframe['plus_dm'] = 0
dataframe['minus_dm'] = 0
dataframe['delta_high'] = dataframe.high - dataframe.high.shift(1)
dataframe['delta_low'] = dataframe.low.shift(1) - dataframe.low
dataframe['delta_high'] = dataframe.delta_high.mask(dataframe.delta_high < 0,0)
dataframe['delta_low'] = dataframe.delta_low.mask(dataframe.delta_low < 0,0)
dataframe['plus_dm'] = dataframe.plus_dm.mask(dataframe.delta_high > dataframe.delta_low,dataframe.delta_high)
dataframe['minus_dm'] = dataframe.minus_dm.mask(dataframe.delta_low > dataframe.delta_high,dataframe.delta_low)

dataframe.at[0,'tr'] = np.nan
dataframe.at[0,'plus_dm'] = np.nan
dataframe.at[0,'minus_dm'] = np.nan

#print(dataframe[['date','high','low','close','tr','plus_dm','minus_dm']])

tr = pd.Series(dataframe.tr)
plus_dm = pd.Series(dataframe.plus_dm)
minus_dm = pd.Series(dataframe.minus_dm)
TRn = pd.Series(np.zeros(tr.size))
plus_DMn = pd.Series(dataframe.plus_dm)
minus_DMn = pd.Series(dataframe.minus_dm)

TRn[TRn.index[n-1]] = np.sum(tr[:n])
TRn = TRn.drop(TRn.index[:n-1])
 
plus_DMn[plus_DMn.index[n-1]] = np.sum(plus_dm[:n])
plus_DMn = plus_DMn.drop(plus_DMn.index[:n-1])


minus_DMn[minus_DMn.index[n-1]] = np.sum(minus_dm[:n])
minus_DMn = minus_DMn.drop(minus_DMn.index[:n-1])

for index,value in plus_DMn.items():
   if index >= n :
       plus_DMn[index] = (plus_DMn[index-1] - (plus_DMn[index-1]/14)) + plus_dm[index]

for index,value in minus_DMn.items():
    if index >= n :
        minus_DMn[index] = (minus_DMn[index-1] - (minus_DMn[index-1]/14)) + minus_dm[index]
for index,value in TRn.items():
    if index >= n :
        TRn[index] = (TRn[index-1] - (TRn[index-1]/14)) + tr[index]

dataframe['plus_DMn'] = plus_DMn
dataframe['minus_DMn'] = minus_DMn
dataframe['TRn'] = TRn

#print(dataframe[['date','tr','plus_dm','minus_dm','TRn','plus_DMn','minus_DMn']])

dataframe['plus_DI'] = dataframe.plus_DMn/dataframe.TRn
dataframe['minus_DI'] = dataframe.minus_DMn/dataframe.TRn

dataframe['TDM'] = abs(dataframe.plus_DI- dataframe.minus_DI)
dataframe['SUM_DM'] = dataframe.plus_DI + dataframe.minus_DI

dataframe['DX'] = dataframe['TDM'] / dataframe['SUM_DM']

#print(dataframe[['plus_DMn','minus_DMn','plus_DI','minus_DI','TDM','SUM_DM','DX']])

dx = pd.Series(dataframe['DX'])
adx = pd.Series(dataframe['DX'])
#print(adx[adx.index[n:2*n]])

adx[adx.index[(2*n) - 1]]= adx[adx.index[n:2*n]].mean()

adx = adx.drop(adx.index[:(2*n)-1])
adx = adx.ewm(com=n-1,adjust=False).mean()
dataframe['ADX'] = adx

dataframe['ADXR'] = (dataframe.ADX + dataframe.ADX.shift(n-1)) / 2
# en realidad deberia ser 14 no 13.

#print(dataframe[['TDM','SUM_DM','DX','ADX',"ADXR"]])

#dataframe.to_csv("faster_final_dataframe_dm" +  str(int(time.time())) +  ".csv")

13 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
def average_directional_movement_index(df, n, n_ADX):
    """Calculate the Average Directional Movement Index for given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :param n_ADX: 
    :return: pandas.DataFrame
    """
    df['tr1'] = df.high - df.low
    df['tr2'] = (df.high - df.close.shift(1)).abs()
    df['tr3'] = (df.low - df.close.shift(1)).abs()
    df['tr'] = df[['tr1','tr2','tr3']].max(axis=1)
    df['plus_dm'] = 0
    df['minus_dm'] = 0
    df['delta_high'] = df.high - df.high.shift(1)
    df['delta_low'] = df.low.shift(1) - df.low
    df['delta_high'] = df.delta_high.mask(df.delta_high < 0,0)
    df['delta_low'] = df.delta_low.mask(df.delta_low < 0,0)
    df['plus_dm'] = df.plus_dm.mask(df.delta_high > df.delta_low,df.delta_high)
    df['minus_dm'] = df.minus_dm.mask(df.delta_low > df.delta_high,df.delta_low)

    df.at[0,'tr'] = np.nan
    df.at[0,'plus_dm'] = np.nan
    df.at[0,'minus_dm'] = np.nan

    #print(dataframe[['date','high','low','close','tr','plus_dm','minus_dm']])

    
    TR_s = pd.Series(dataframe.tr)
    ATR = pd.Series(TR_s.ewm(span=n, min_periods=n).mean())
    UpI = pd.Series(df.plus_dm)
    DoI = pd.Series(df.minus_dm)
    PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean() / ATR)
    NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean() / ATR)
    ADX = pd.Series((abs(PosDI - NegDI) / (PosDI + NegDI)).ewm(span=n_ADX, min_periods=n_ADX).mean(),
                    name='ADX_' + str(n) + '_' + str(n_ADX))
    df = df.join(ADX)
    return df

In [73]:
%%timeit
result = average_directional_movement_index(dataframe,14,14)

8.71 ms ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
def original_average_directional_movement_index(df, n, n_ADX):
    """Calculate the Average Directional Movement Index for given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :param n_ADX: 
    :return: pandas.DataFrame
    """
    i = 0
    UpI = []
    DoI = []
    while i + 1 <= df.index[-1]:
        UpMove = df.loc[i + 1, 'high'] - df.loc[i, 'high']
        DoMove = df.loc[i, 'low'] - df.loc[i + 1, 'low']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i = i + 1
    i = 0
    TR_l = [0]
    while i < df.index[-1]:
        TR = max(df.loc[i + 1, 'high'], df.loc[i, 'close']) - min(df.loc[i + 1, 'low'], df.loc[i, 'close'])
        TR_l.append(TR)
        i = i + 1
    TR_s = pd.Series(TR_l)
    ATR = pd.Series(TR_s.ewm(span=n, min_periods=n).mean())
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean() / ATR)
    NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean() / ATR)
    ADX = pd.Series((abs(PosDI - NegDI) / (PosDI + NegDI)).ewm(span=n_ADX, min_periods=n_ADX).mean(),
                    name='ADX_' + str(n) + '_' + str(n_ADX))
    df = df.join(ADX)
    return df

In [70]:
%%timeit
result = original_average_directional_movement_index(dataframe,14,14)

5.89 ms ± 38 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
